In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [ ]:
# Cell 1: library imports
import json
import numpy as np
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from datetime import datetime
import os

/home/wonjun/.conda/envs/mlproject/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Cell 2: 경로 설정
# 경로 설정
DATA_DIR = Path("guardian_scrapping")  # JSONL 파일들이 있는 디렉토리
OUTPUT_DIR = Path("./miniLM_headline_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# 출력 파일 경로
EMBEDDINGS_FILE = OUTPUT_DIR / "embeddings.npy"
METADATA_FILE = OUTPUT_DIR / "metadata.jsonl" 
CHECKPOINT_FILE = OUTPUT_DIR / "checkpoint.txt"

# 모델 로드 (all_miniLM)
print("Loading Sentence-BERT model...")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
print(f"Model loaded! Embedding dimension: {model.get_sentence_embedding_dimension()}")

Loading Sentence-BERT model...
Model loaded! Embedding dimension: 384


In [4]:
# Cell 3: JSONL 파일 찾기 및 people_list 생성
# 모든 JSONL 파일 찾기 (scrapping_csv 폴더 제외)
jsonl_files = [f for f in DATA_DIR.glob("*.jsonl")]

# people_list 생성 (파일명에서 .jsonl 제거)
people_list = sorted([f.stem for f in jsonl_files])

print(f"Found {len(people_list)} people:")
print(people_list[:10], "..." if len(people_list) > 10 else "")

Found 100 people:
['adam_neumann', 'adam_rippon', 'alex_wind', 'ann_mckee', 'bhavish_aggarwal', 'cameron_kasky', 'cardi_b', 'carl_june', 'carmen_yulin_cruz', 'chadwick_boseman'] ...


In [5]:
# Cell 4: 이어달리기 기능 - 이미 처리된 파일 확인
"""
이어달리기 로직 설명:
1. checkpoint.txt 파일에 마지막으로 처리한 사람의 이름을 저장
2. 프로그램 재시작 시 checkpoint.txt를 읽어서 어디까지 처리했는지 확인
3. 이미 처리된 사람은 건너뛰고, 그 다음 사람부터 처리
4. embeddings.npy와 metadata.jsonl은 append 모드로 저장
"""

def get_last_processed():
    """마지막으로 처리한 사람 확인"""
    if CHECKPOINT_FILE.exists():
        with open(CHECKPOINT_FILE, 'r') as f:
            return f.read().strip()
    return None

def save_checkpoint(person_name):
    """체크포인트 저장"""
    with open(CHECKPOINT_FILE, 'w') as f:
        f.write(person_name)

def load_existing_data():
    """기존에 저장된 데이터 로드"""
    if EMBEDDINGS_FILE.exists() and METADATA_FILE.exists():
        embeddings = np.load(EMBEDDINGS_FILE)
        
        # JSONL 파일 읽기
        metadata = []
        with open(METADATA_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                metadata.append(json.loads(line))
        
        return embeddings, metadata
    return None, None

# 기존 데이터 로드
existing_embeddings, existing_metadata = load_existing_data()
last_processed = get_last_processed()

if last_processed:
    print(f"✅ Resuming from: {last_processed}")
    print(f"✅ Already processed {len(existing_metadata) if existing_metadata else 0} articles")
else:
    print("🆕 Starting fresh")

🆕 Starting fresh


In [6]:
# Cell 5: 날짜 변환 함수
def format_date(date_string):
    """2017-12-21T08:00:01Z -> 2017_12_21"""
    if not date_string:
        return None
    try:
        return date_string.split('T')[0].replace('-', '_')
    except:
        return None

In [7]:
# Cell 6: 메인 처리 함수
def process_person(person_name, jsonl_path, start_index):
    """한 사람의 JSONL 파일을 처리하여 임베딩 생성"""
    articles_data = []
    
    # JSONL 파일 읽기
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                article = json.loads(line)
                
                # 필수 필드 확인 (결측치 제거)
                if not article.get('headline') or not article.get('id'):
                    continue
                
                # 데이터 추출
                articles_data.append({
                    'person': person_name,
                    'article_id': article['id'],
                    'pub_date': format_date(article.get('webPublicationDate')),
                    'headline': article['headline']
                })
            except:
                continue  # 파싱 에러 발생 시 건너뛰기
    
    if not articles_data:
        return None, None
    
    # 임베딩 생성
    headlines = [item['headline'] for item in articles_data]
    embeddings = model.encode(headlines, show_progress_bar=False, convert_to_numpy=True)
    
    # index 추가
    for i, item in enumerate(articles_data):
        item['index'] = start_index + i
    
    return embeddings, articles_data

In [8]:
# Cell 7: 전체 처리 실행
"""
전체 프로세스:
1. people_list를 순회하면서 각 사람의 JSONL 파일 처리
2. 이어달리기: last_processed 이후의 사람들만 처리
3. 각 사람마다 임베딩과 메타데이터 생성
4. 누적하여 저장하고 체크포인트 업데이트
"""

# 기존 데이터로 시작
all_embeddings = [] if existing_embeddings is None else [existing_embeddings]
all_metadata = [] if existing_metadata is None else existing_metadata

# 현재 인덱스 (이어달리기)
current_index = len(all_metadata)

# 시작 인덱스 결정 (이어달리기)
start_idx = 0
if last_processed and last_processed in people_list:
    start_idx = people_list.index(last_processed) + 1

# 처리할 사람들
people_to_process = people_list[start_idx:]

print(f"\n🚀 Processing {len(people_to_process)} people (from {start_idx}/{len(people_list)})")
print("=" * 70)

# 각 사람별로 처리
for person_name in tqdm(people_to_process, desc="Overall Progress"):
    jsonl_path = DATA_DIR / f"{person_name}.jsonl"
    
    if not jsonl_path.exists():
        print(f"⚠️  File not found: {jsonl_path}")
        continue
    
    #print(f"\n📄 Processing: {person_name}")
    
    # 임베딩 생성
    embeddings, metadata = process_person(person_name, jsonl_path, current_index)
    
    if embeddings is None:
        print(f"   ⚠️  No valid articles found for {person_name}")
        continue
    
    #print(f"   ✅ Generated {len(embeddings)} embeddings")
    
    # 데이터 누적
    all_embeddings.append(embeddings)
    all_metadata.extend(metadata)
    current_index += len(embeddings)
    
    # 중간 저장 (메모리 효율)
    if len(all_embeddings) % 10 == 0:  # 10명마다 저장
        # 임베딩 저장
        combined_embeddings = np.vstack(all_embeddings)
        np.save(EMBEDDINGS_FILE, combined_embeddings)
        
        # JSONL 저장
        with open(METADATA_FILE, 'w', encoding='utf-8') as f:
            for item in all_metadata:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        
        print(f"   💾 Checkpoint saved: {len(combined_embeddings)} total embeddings")
    
    # 체크포인트 업데이트
    save_checkpoint(person_name)

print("\n" + "=" * 70)
print("🎉 Processing complete!")


🚀 Processing 100 people (from 0/100)


Overall Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Overall Progress:  10%|█         | 10/100 [00:06<01:03,  1.42it/s]

   💾 Checkpoint saved: 9422 total embeddings


Overall Progress:  20%|██        | 20/100 [00:33<05:04,  3.80s/it]

...

   💾 Checkpoint saved: 190393 total embeddings


Overall Progress:  90%|█████████ | 90/100 [02:21<00:22,  2.21s/it]

   💾 Checkpoint saved: 215186 total embeddings


Overall Progress: 100%|██████████| 100/100 [02:36<00:00,  1.56s/it]

   💾 Checkpoint saved: 232583 total embeddings

🎉 Processing complete!


In [9]:
# Cell 8: 최종 저장
# 모든 임베딩 결합
final_embeddings = np.vstack(all_embeddings)
np.save(EMBEDDINGS_FILE, final_embeddings)

# JSONL 최종 저장
with open(METADATA_FILE, 'w', encoding='utf-8') as f:
    for item in all_metadata:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"\n✅ Final Results:")
print(f"   📊 Total embeddings: {final_embeddings.shape}")
print(f"   📊 Total articles: {len(all_metadata)}")
print(f"   💾 Embeddings saved to: {EMBEDDINGS_FILE}")
print(f"   💾 Metadata saved to: {METADATA_FILE}")


✅ Final Results:
   📊 Total embeddings: (232583, 384)
   📊 Total articles: 232583
   💾 Embeddings saved to: miniLM_headline_outputs/embeddings.npy
   💾 Metadata saved to: miniLM_headline_outputs/metadata.jsonl


In [ ]:
# Cell 9: 결과 확인
# 임베딩 확인
embeddings = np.load(EMBEDDINGS_FILE)
print(f"Embeddings shape: {embeddings.shape}")
print(f"First embedding (first 10 dims): {embeddings[0][:10]}")

# JSONL 메타데이터 확인
metadata = []
with open(METADATA_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        metadata.append(json.loads(line))

print(f"\nMetadata count: {len(metadata)}")
print(f"\nFirst 3 items:")
for item in metadata[:3]:
    print(json.dumps(item, indent=2, ensure_ascii=False))

# 사람별 분포
from collections import Counter
person_counts = Counter(item['person'] for item in metadata)
print(f"\nTop 10 people:")
for person, count in person_counts.most_common(10):
    print(f"  {person}: {count} articles")

Embeddings shape: (232583, 384)
First embedding (first 10 dims): [-0.04351391 -0.02885366 -0.01260174 -0.01757218 -0.02314188  0.04706983
  0.02907552  0.01892885  0.00625776 -0.06741666]

Metadata count: 232583

First 3 items:
{
  "person": "adam_neumann",
  "article_id": "lifeandstyle/2017/dec/27/adam-gopnik-miracle-before-christmas",
  "pub_date": "2017_12_27",
  "headline": "Adam Gopnik: child star of avant-garde theatre on how a snowstorm saved Christmas",
  "index": 0
}
{
  "person": "adam_neumann",
  "article_id": "us-news/2017/dec/16/republicans-trying-to-kill-off-trump-russia-investigation-says-adam-schiff",
  "pub_date": "2017_12_16",
  "headline": "Trump-Russia: Republicans trying to kill off investigation, says Adam Schiff",
  "index": 1
}
{
  "person": "adam_neumann",
  "article_id": "books/2017/dec/10/adam-kay-this-is-going-to-hurt-interview-junior-doctor",
  "pub_date": "2017_12_10",
  "headline": "Adam Kay: ‘If I had kids I would put them off studying medicine’",
  "ind